# 04_Domain_Adaptation_FineTuning.ipynb

## Descripción General
Este cuaderno aborda el desafío de la **Brecha de Dominio (Domain Gap)** existente entre el dataset de entrenamiento académico (CIFAR-10, 32x32px) y las imágenes del mundo real (HD).

Aunque el modelo base ha alcanzado métricas de SOTA en el entorno controlado, su rendimiento puede degradarse ante texturas de alta frecuencia y artefactos de compresión presentes en fotografías reales.

## Estrategia de Adaptación
1.  **Ingesta de Datos HD:** Carga de un dataset curado de imágenes de alta resolución (`dataset_hd`).
2.  **Alineación de Tensores:** Verificación y corrección automática del orden de las clases (One-Hot Mapping) para garantizar la consistencia con la topología del modelo base.
3.  **Fine-Tuning Conservador:** Entrenamiento con una tasa de aprendizaje reducida (`1e-5`) para ajustar los pesos de las capas profundas sin destruir la extracción de características aprendida previamente (Catastrophic Forgetting).
4.  **Exportación a Producción:** Generación del binario final optimizado para inferencia.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import gc

# 1. Gestion de Recursos
tf.keras.backend.clear_session()
gc.collect()

# 2. Configuracion del Entorno
IMG_SIZE = 224
BATCH_SIZE = 8  # Batch reducido para estabilidad durante el ajuste fino
EPOCHS_FT = 15  # Ciclo corto de epocas
LEARNING_RATE = 1e-5 # Tasa de aprendizaje de magnitud reducida

# Directorios de Trabajo
BASE_DIR = '/tf/notebooks'
HD_DATA_DIR = os.path.join(BASE_DIR, 'dataset_hd')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
PROD_DIR = os.path.join(BASE_DIR, 'production_models')
OUTPUTS_DIR = os.path.join(BASE_DIR, 'outputs')

# Orden estricto de clases esperado por el modelo base (CIFAR-10 Mapping)
# Indice 0: Dog, Indice 1: Automobile, Indice 2: Bird
MODEL_CLASS_ORDER = ['dog', 'automobile', 'bird']

print("Entorno de Fine-Tuning inicializado.")

## 1. Carga y Sincronización de Datos HD
La carga de datos desde directorios (`image_dataset_from_directory`) asigna índices basados en el orden alfabético de las carpetas. Esto suele diferir del orden lógico definido manualmente en etapas anteriores.

Se implementa una capa de preprocesamiento lógico que detecta la discrepancia y reordena los tensores de etiquetas (One-Hot) dinámicamente para evitar la corrupción del conocimiento del modelo (ej. evitar que aprenda que "Auto" ahora es "Perro").

In [ ]:
# ============================================================================
# 1. CARGA DE DATASET HD Y ALINEACION
# ============================================================================

print("Cargando dataset de Adaptacion (HD)...")

# Carga inicial (Orden alfabetico predeterminado)
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    HD_DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    HD_DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Deteccion del orden actual en disco
detected_order = train_ds_raw.class_names
print(f"\nOrden detectado en disco: {detected_order}")
print(f"Orden requerido por modelo: {MODEL_CLASS_ORDER}")

# Logica de reordenamiento de etiquetas
if detected_order != MODEL_CLASS_ORDER:
    print("ALERTA: Discrepancia de orden detectada. Aplicando correccion de tensores...")
    
    # Calculo de indices de permutacion
    # Mapea donde esta cada clase requerida en la lista detectada
    perm_indices = [detected_order.index(cls) for cls in MODEL_CLASS_ORDER]
    
    def fix_label_order(images, labels):
        """Reordena las columnas del vector One-Hot segun la permutacion calculada"""
        return images, tf.gather(labels, perm_indices, axis=1)
    
    # Aplicacion al pipeline
    train_ds = train_ds_raw.map(fix_label_order)
    val_ds = val_ds_raw.map(fix_label_order)
    print("Correccion aplicada exitosamente.")
else:
    train_ds = train_ds_raw
    val_ds = val_ds_raw
    print("El orden coincide. No se requieren correcciones.")

# Preprocesamiento final (Normalizacion ConvNeXt)
from tensorflow.keras.applications.convnext import preprocess_input

def preprocess_input_layer(images, labels):
    return preprocess_input(images), labels

train_ds = train_ds.map(preprocess_input_layer)
val_ds = val_ds.map(preprocess_input_layer)

## 2. Evaluación de Línea Base (Baseline)
Antes de modificar los pesos del modelo, se evalúa su rendimiento actual sobre el dataset HD. Esto establece un punto de partida para cuantificar la mejora obtenida mediante la adaptación de dominio.

In [ ]:
# ============================================================================
# 2. CARGA DEL MODELO BASE Y BENCHMARK
# ============================================================================

# Identificacion del ultimo checkpoint
list_of_models = glob.glob(os.path.join(MODELS_DIR, 'best_model_*.keras'))
if not list_of_models:
    raise FileNotFoundError("No se encontraron modelos base en models/")

latest_model_path = max(list_of_models, key=os.path.getctime)
print(f"Cargando modelo base: {os.path.basename(latest_model_path)}")

model = keras.models.load_model(latest_model_path)

# Evaluacion
print("\nEjecutando evaluacion de linea base (Pre-Tuning)...")
baseline_results = model.evaluate(val_ds, verbose=0)
print(f"Accuracy Inicial (HD): {baseline_results[1]*100:.2f}%")
print(f"Loss Inicial (HD):     {baseline_results[0]:.4f}")

## 3. Ejecución de Fine-Tuning
Se procede al re-entrenamiento del modelo.
* **Optimizador:** Adam.
* **Learning Rate:** `1e-5`. Este valor es crítico; un valor más alto podría destruir los patrones aprendidos en CIFAR-10.
* **Capas:** Se mantienen todas las capas entrenables (*Unfrozen*) para permitir una adaptación global de los mapas de características a la nueva resolución y texturas.

In [ ]:
# ============================================================================
# 3. FINE-TUNING (ADAPTACION DE DOMINIO)
# ============================================================================

print(f"Iniciando Fine-Tuning ({EPOCHS_FT} epocas, LR={LEARNING_RATE})...")

model.compile(
    optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FT,
    verbose=1
)

# Evaluacion Post-Tuning
final_results = model.evaluate(val_ds, verbose=0)
improvement = (final_results[1] - baseline_results[1]) * 100

print("\nRESULTADOS DE LA ADAPTACION:")
print(f"Accuracy Final (HD): {final_results[1]*100:.2f}%")
print(f"Mejora Absoluta:     +{improvement:.2f}%")

## 4. Validación de Calidad
Generación de métricas detalladas utilizando el conjunto de validación HD para asegurar que el modelo no ha introducido sesgos hacia ninguna clase específica durante la adaptación.

In [ ]:
# ============================================================================
# 4. ANALISIS DE RESULTADOS
# ============================================================================

# Extraccion de etiquetas y predicciones
val_images = []
val_labels = []

for img, label in val_ds:
    val_images.append(img.numpy())
    val_labels.append(label.numpy())

X_val_np = np.vstack(val_images)
y_val_np = np.vstack(val_labels)

# Inferencia
y_pred_probs = model.predict(X_val_np, verbose=0)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_val_np, axis=1)

# Matriz de Confusion
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=MODEL_CLASS_ORDER,
            yticklabels=MODEL_CLASS_ORDER)
plt.title('Matriz de Confusión: Modelo Adaptado HD')
plt.ylabel('Etiqueta Real')
plt.xlabel('Predicción')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUTS_DIR, 'confusion_matrix_hd.png'))
plt.show()

print("\nReporte de Clasificacion:")
print(classification_report(y_true_classes, y_pred_classes, target_names=MODEL_CLASS_ORDER))

## 5. Despliegue
Exportación del modelo final adaptado (`model_production_hd.keras`). Este artefacto reemplazará al modelo anterior en el entorno de producción (FastAPI).

In [ ]:
# ============================================================================
# 5. EXPORTACION A PRODUCCION
# ============================================================================

os.makedirs(PROD_DIR, exist_ok=True)
final_path = os.path.join(PROD_DIR, 'model_production_hd.keras')

print(f"Guardando modelo optimizado en: {final_path}")

# Guardado sin optimizador para reducir tamano (~50%)
model.save(final_path, include_optimizer=False)

print("Proceso completado. Modelo listo para integracion en API.")